#### Server Side Version Counters

The `version_id_generator` can also be _configured to rely upon_ a value that is __generated by the database__. In this case, the database would need some means of __generating new identifiers__ when a row is _subject to an INSERT_ as well as with an _UPDATE_. For the _UPDATE_ case, typically an _update trigger_ is needed, unless the database in question supports some other __native version identifier__. The _PostgreSQL_ database in particular supports a _system column_ called `xmin` which __provides UPDATE versioning__. We can make use of the _PostgreSQL_ `xmin` column to version our _User_ class as follows.

In [1]:
from sqlalchemy import Column, FetchedValue, Integer, String
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    xmin = Column("xmin", String, system=True, server_default=FetchedValue())
    
    __mapper_args__ = {"version_id_col": xmin, "version_id_generator": False}

With the above mapping, the ORM will rely upon the _xmin_ column for __automatically__ providing the _new value of the version id counter_.

> ##### creating tables that refer to system columns
>
> In the above scenario, as _xmin_ is a __system column__ provided by PostgreSQL, we use the `system=True` argument to _mark it as a system-provided column_, __omitted from the `CREATE TABLE` statement__. The _datatype_ of this column is an _internal PostgreSQL type_ called `xid` which acts mostly __like a string__, so we use the `String` datatype.

The ORM typically __does not actively fetch__ the values of _database-generated_ values when it _emits_ an `INSERT` or `UPDATE`, instead leaving these columns as `"expired"` and to be fetched when they are _next accessed_, unless the __eager_defaults `mapper()` flag__ is set. However, when a _server side version column_ is used, the ORM __needs to actively fetch__ the _newly generated value_. This is so that the _version counter_ is __set up before any concurrent transaction may update it again__. This fetching is also __best done simultaneously within the `INSERT` or `UPDATE` statement using `RETURNING`__, otherwise if _emitting a SELECT statement afterwards_, there is still a __potential `race condition`__ where the _version counter_ __may change before it can be fetched__.

When the target database supports _RETURNING_, an _INSERT_ statement for our _User_ class will look like this:

```
INSERT INTO "user" (name) VALUES (%(name)s) RETURNING "user".id, "user".xmin
{'name': 'ed'}
```

Where above, the ORM __can acquire__ any _newly generated primary key values_ along with _server-generated version identifiers_ in one statement. When the __backend does not support `RETURNING`__, an __additional `SELECT` must be emitted__ for every _INSERT_ and _UPDATE_, which is __much less efficient__, and also introduces the _possibility of missed version counters_:

```
INSERT INTO "user" (name) VALUES (%(name)s)
{'name': 'ed'}

SELECT "user".version_id AS user_version_id FROM "user" where
"user".id = :param_1
{"param_1": 1}
```

It is __strongly recommended__ that _server side version counters_ only be used when __absolutely necessary__ and __only on backends that support `RETURNING`__, e.g. _PostgreSQL_, _Oracle_, _SQL Server_ (though SQL Server has major caveats when triggers are used), _Firebird_.